<a href="https://colab.research.google.com/github/202248SD/5DayGitChallenge/blob/main/ClusterAndLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/犬１-20240617T025624Z-001.zip
!unzip /content/犬２-20240617T025639Z-001.zip

In [10]:
!rm /content/犬１-20240617T025624Z-001.zip
!rm /content/犬２-20240617T025639Z-001.zip

In [7]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from PIL import Image
import os
import torchvision.models as models
import torch.nn as nn
from glob import glob

In [6]:
model = models.resnet101(weights='DEFAULT')

model.eval()

modules = list(model.children())[:-1]
feature_extractor = nn.Sequential(*modules)

preprocess = transforms.Compose([
  transforms.Resize(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image_path):
  img = Image.open(image_path)
  img_t = preprocess(img)
  batch_t = torch.unsqueeze(img_t, 0)

  with torch.no_grad():
      features = feature_extractor(batch_t)
  return features.squeeze()

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:01<00:00, 89.6MB/s]


In [16]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [17]:
from ultralytics import YOLO
DogDetector = YOLO("yolov8l.pt")

100%|██████████| 83.7M/83.7M [00:00<00:00, 270MB/s]


In [ ]:
DogTypeNet = models.resnet50(pretrained=True)
num_ftrs = DogTypeNet.fc.in_features
DogTypeNet.fc = torch.nn.Linear(num_ftrs, 120)
DogTypeNet.load_state_dict(torch.load('/content/model (2).pth', map_location=torch.device('cpu')))
idx_to_class = {0: 'Chihuahua', 1: 'Japanese_spaniel', 2: 'Maltese_dog', 3: 'Pekinese', 4: 'Shih-Tzu', 5: 'Blenheim_spaniel', 6: 'papillon', 7: 'toy_terrier', 8: 'Rhodesian_ridgeback', 9: 'Afghan_hound', 10: 'basset', 11: 'beagle', 12: 'bloodhound', 13: 'bluetick', 14: 'black-and-tan_coonhound', 15: 'Walker_hound', 16: 'English_foxhound', 17: 'redbone', 18: 'borzoi', 19: 'Irish_wolfhound', 20: 'Italian_greyhound', 21: 'whippet', 22: 'Ibizan_hound', 23: 'Norwegian_elkhound', 24: 'otterhound', 25: 'Saluki', 26: 'Scottish_deerhound', 27: 'Weimaraner', 28: 'Staffordshire_bullterrier', 29: 'American_Staffordshire_terrier', 30: 'Bedlington_terrier', 31: 'Border_terrier', 32: 'Kerry_blue_terrier', 33: 'Irish_terrier', 34: 'Norfolk_terrier', 35: 'Norwich_terrier', 36: 'Yorkshire_terrier', 37: 'wire-haired_fox_terrier', 38: 'Lakeland_terrier', 39: 'Sealyham_terrier', 40: 'Airedale', 41: 'cairn', 42: 'Australian_terrier', 43: 'Dandie_Dinmont', 44: 'Boston_bull', 45: 'miniature_schnauzer', 46: 'giant_schnauzer', 47: 'standard_schnauzer', 48: 'Scotch_terrier', 49: 'Tibetan_terrier', 50: 'silky_terrier', 51: 'soft-coated_wheaten_terrier', 52: 'West_Highland_white_terrier', 53: 'Lhasa', 54: 'flat-coated_retriever', 55: 'curly-coated_retriever', 56: 'golden_retriever', 57: 'Labrador_retriever', 58: 'Chesapeake_Bay_retriever', 59: 'German_short-haired_pointer', 60: 'vizsla', 61: 'English_setter', 62: 'Irish_setter', 63: 'Gordon_setter', 64: 'Brittany_spaniel', 65: 'clumber', 66: 'English_springer', 67: 'Welsh_springer_spaniel', 68: 'cocker_spaniel', 69: 'Sussex_spaniel', 70: 'Irish_water_spaniel', 71: 'kuvasz', 72: 'schipperke', 73: 'groenendael', 74: 'malinois', 75: 'briard', 76: 'kelpie', 77: 'komondor', 78: 'Old_English_sheepdog', 79: 'Shetland_sheepdog', 80: 'collie', 81: 'Border_collie', 82: 'Bouvier_des_Flandres', 83: 'Rottweiler', 84: 'German_shepherd', 85: 'Doberman', 86: 'miniature_pinscher', 87: 'Greater_Swiss_Mountain_dog', 88: 'Bernese_mountain_dog', 89: 'Appenzeller', 90: 'EntleBucher', 91: 'boxer', 92: 'bull_mastiff', 93: 'Tibetan_mastiff', 94: 'French_bulldog', 95: 'Great_Dane', 96: 'Saint_Bernard', 97: 'Eskimo_dog', 98: 'malamute', 99: 'Siberian_husky', 100: 'affenpinscher', 101: 'basenji', 102: 'pug', 103: 'Leonberg', 104: 'Newfoundland', 105: 'Great_Pyrenees', 106: 'Samoyed', 107: 'Pomeranian', 108: 'chow', 109: 'keeshond', 110: 'Brabancon_griffon', 111: 'Pembroke', 112: 'Cardigan', 113: 'toy_poodle', 114: 'miniature_poodle', 115: 'standard_poodle', 116: 'Mexican_hairless', 117: 'dingo', 118: 'dhole', 119: 'African_hunting_dog'}

In [18]:
import numpy as np
from glob import glob
inu_1 = np.array(glob("犬１/*"))
inu_2 = np.array(glob("犬２/*"))

In [20]:
def process_image(img_path, x_min, y_min, x_max, y_max):
    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = Image.open(img_path).convert('RGB')

    image = image.crop((x_min, y_min, x_max, y_max))

    image = preprocess(image)

    image = image.unsqueeze(0)

    return image

In [ ]:
def cluster_images(img_paths, n_clusters):
  features = [] # [img_path, ftrs1, ftrs2, ...]
  labels = [] # [img_path, label1, label2, ...]
  # label = [name, x_min, y_min, x_max, y_max]
  for img_path in img_paths:
    results = DogDetector(img_path, conf=0.3, iou=0.3)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()

    img = cv.imread(img_path)
    img_features = [img_path]
    img_labels = [img_path]

    for box, cls, conf in zip(boxes, classes, confidences):
      if cls not in [0, 16]:
        continue
      else:
        x_min, y_min, x_max, y_max = [int(i) for i in box]
        name = names[int(cls)]
        if cls == 16:
          box_img = process_image(img_path, x_min, y_min, x_max, y_max)
          img_features.append(extract_features(box_img).numpy())
          name = DogTypeNet(box_img)
        img_labels.append(name
        else:


    img_features = np.array(img_features)
    features.append(img_features)

  features = np.array(features)
  pca = PCA(n_components=128)
  pca.fit(features)
  pca_features = pca.transform(features)

  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  kmeans.fit(pca_features)

  return kmeans.labels_